### ORAS5

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import function
import matplotlib.colors as mcolors
from netCDF4 import Dataset
def mld_avg_var(mld,var,z):
    zmask= (z < mld) # z above mixed layer
    intfield=var.where(zmask, drop=True).mean('LEV').squeeze()
    return intfield
'''统一经纬度，用xarray来显示net_flux'''
latitude=np.arange(0.0, 20.30, 0.247)
longitude=np.arange(40.0, 65.25, 0.25)
d_temp=-0.14
file="E:\data\oras5\somali-oras5\oras5_1979-2023_ptemp_500m.nc";
ds= xr.open_dataset(file)
ds= list(ds.data_vars.values())[0]
ds=ds.sel(LEV=slice(0,120)).groupby('time.month').mean('time').rename({'month': 'time'})
sst=ds.sel(LEV=ds.LEV[7])
ds=ds.interp(LEV=np.arange(ds.LEV[0],120,0.5))#这一步非常关键
a=(ds>(sst+d_temp))
ild_idx=ds.where(a,drop=False).squeeze().notnull().argmin(dim='LEV')#
ild=ds.LEV[ild_idx];print('max:',ild.max())

In [ ]:
mask=function.make_mask3D(sst,5/(ds.latitude.data[1]-ds.latitude.data[0]),sst)*0

In [ ]:
d_rho=0.15
file="E:\\data\\oras5\\somali-oras5\\time_series\\oras5_opa4_rho_1979-2023.nc";ds= xr.open_dataset(file);ds= list(ds.data_vars.values())[0]
ds=ds.groupby('time.month').mean('time').rename({'month': 'time'})
ds=ds-ds.mean()
ssrho=ds.sel(LEV=ds.LEV[0])
ds=ds.interp(LEV=np.arange(ds.LEV[0],120,0.5))
a=(ds<(ssrho+d_rho))
mld_idx=ds.where(a,drop=False).squeeze().notnull().argmin(dim='LEV')#寻找最大深度，notnull使得缺失值为False，不参与后面的计算
mld=ds.LEV[mld_idx];print('max:',mld.max())

In [ ]:
file="E:\data\oras5\somali-oras5\oras5_1979-2023_ptemp_500m.nc"
ds0 = xr.open_dataset(file);ds= list(ds0.data_vars.values())[0]#选定变量
ds=ds.groupby('time.month').mean('time').rename({'month': 'time'}).sel(LEV=slice(0,200))
ptemp=ds.interp(LEV=np.arange(ds.LEV[0],200,0.5))
mlt=mld_avg_var(mld,ptemp,ptemp.LEV)
levels=np.arange(18 , 32, 0.5);colors='jet'
file="E:\data\oras5\somali-oras5\oras5_mld&ild&mlt&T_n&mlu&mlv_climate.nc"
ds0 = xr.open_dataset(file);mlt1 = ds0['mlt'].sel(latitude=slice(0,20))

In [ ]:
ds=ptemp.interp(LEV=np.arange(ptemp.LEV[0],120,0.5))
a=(ds.LEV==ds.LEV[mld_idx]+5)
T_n=ds.where(a, drop=False).mean('LEV').squeeze()
levels=np.arange(18 , 32, 0.5);colors='jet'
file="E:\data\oras5\somali-oras5\oras5_mld&ild&mlt&T_n&mlu&mlv_climate.nc"
ds0 = xr.open_dataset(file);mlt1 = ds0['T_n'].sel(latitude=slice(0,20))

In [ ]:
file="E:\data\oras5\somali-oras5\oras5_u_current_climate_200m.nc.nc"
ds0 = xr.open_dataset(file);ds= list(ds0.data_vars.values())[0]#选定变量
u=ds.interp(LEV=np.arange(ds.LEV[0],200,0.5))
mlu=mld_avg_var(mld,u,u.LEV)
levels=np.arange(-1,1, 0.1);colors='seismic'
file="E:\data\oras5\somali-oras5\oras5_mld&ild&mlt&T_n&mlu&mlv_climate.nc"
ds0 = xr.open_dataset(file);mlt1 = ds0['mlu'].sel(latitude=slice(0,20))

In [ ]:
file="E:\data\oras5\somali-oras5\oras5_v_current_climate_200m.nc.nc"
ds0 = xr.open_dataset(file);ds= list(ds0.data_vars.values())[0]#选定变量
v=ds.interp(LEV=np.arange(ds.LEV[0],200,0.5))
mlv=mld_avg_var(mld,v,v.LEV)
levels=np.arange(-1,1, 0.1);colors='seismic'
file="E:\data\oras5\somali-oras5\oras5_mld&ild&mlt&T_n&mlu&mlv_climate.nc"
ds0 = xr.open_dataset(file);mlt1 = ds0['mlv'].sel(latitude=slice(0,20))

In [ ]:
def calc_qpen(SW,MLD):
    eu=2.71818
    T1=0.58*eu**(-MLD/0.35)
    T2=0.42*eu**(-MLD/23)
    qpen=SW*(T1+T2)
    return qpen
file1="E:\data\era5\era5_40~80E_uwind_vwind_at_evap_lhf_lwr_swr_shf_sst-1980-2023.nc";ds1=xr.open_dataset(file1);ds=ds1.msnswrf
ds=ds.groupby('time.month').mean().rename({'month': 'time'});ds=ds.reindex(latitude=list(reversed(ds.latitude)))
sw=ds.interp(latitude=mld.latitude.values, longitude=mld.longitude.values);sw=sw.sel(latitude=slice(-0.5,20.5));
Qpen=calc_qpen(sw,mld)
file1="E:\\data\\oras5\\somali-oras5\\no_cycle_MLD_every_term_climate.nc";ds1=xr.open_dataset(file1);ds=ds1['Qpen']

In [ ]:
rhoconst = 1024# Seawater density (kg/m^3)
c_p = 3994# Heat capacity (J/kg/K)
const=30*24*60*60/(rhoconst*c_p)
file1="E:\data\oras5\somali-oras5\oras5_1979-2023_qnet.nc";ds = xr.open_dataset(file1);ds= list(ds.data_vars.values())[0]
ds=ds.groupby('time.month').mean('time').rename({'month': 'time'});Qnet=ds
file="E:\\data\\oras5\\somali-oras5\\no_cycle_MLD_every_term_climate.nc";ds0 = xr.open_dataset(file);Qpen1=ds0.Qpen
Q1=(Qnet-Qpen1)*const/mld.data
Q=(Qnet-Qpen)*const/mld.data

In [ ]:
a=ptemp.where(ptemp>24).notnull().argmin(dim='LEV')
He_24=(ptemp.LEV[a])
file1="E:\\data\\oras5\\somali-oras5\\no_cycle_MLD_every_term_climate.nc";ds1=xr.open_dataset(file1);ds=ds1['He_24']

In [ ]:
file1="E:\data\era5\era5_40~80E_uwind_vwind_at_evap_lhf_lwr_swr_shf_sst-1980-2023.nc";ds1=xr.open_dataset(file1);ds=ds1.msnswrf
time=ds.time[0:12];latitude=mlt.latitude;longitude=mlt.longitude
sal= xr.DataArray(mlt.data, coords=[time,latitude,longitude],dims=['time', 'lat','lon']);lat=sal.lat;lon=sal.lon;
int_U= xr.DataArray(mlu, coords=[time,latitude,longitude],dims=['time', 'lat','lon'])
int_V= xr.DataArray(mlv, coords=[time,latitude,longitude],dims=['time', 'lat','lon'])
from advection_rd_ml import advection_rd_ml
mnadv_climU,mnadv_climV,ekman,T1,h_eddy=advection_rd_ml(sal,lat,lon,time,int_U,int_V,1,-1)
adv_u=mnadv_climU.rename({'month': 'time','lat':'latitude','lon':'longitude'})*86400*30.41;
adv_v=mnadv_climV.rename({'month': 'time','lat':'latitude','lon':'longitude'})*86400*30.41
mask1=function.make_mask3D(adv_u,abs(5/(ds.latitude.data[1]-ds.latitude.data[0])),adv_u)

In [ ]:
file="E:\data\oras5\somali-oras5\oras5_mld&ild&mlt&T_n&mlu&mlv_climate.nc"
ds0 = xr.open_dataset(file)
mlu_seasonal_means = ds0['mlu']
mlv_seasonal_means = ds0['mlv']
mlt_seasonal_means = ds0['mlt']

key=mlv_seasonal_means.time.data
df=mlt_seasonal_means
mlt_seasonal_means=df.fillna(mlt_seasonal_means.mean())+mlt_seasonal_means.data[0,:,:]*0

In [ ]:
def concat_time(dat):
    dat=xr.DataArray(dat.data, coords=[dat.time,dat.latitude,dat.longitude],dims=['time', 'latitude','longitude'])
    front=xr.DataArray(dat.sel(time=dat.time[-1:]).data, coords=[[0],dat.latitude,dat.longitude],dims=['time', 'latitude','longitude'])
    behind=xr.DataArray(dat.sel(time=dat.time[:1]).data, coords=[[13],dat.latitude,dat.longitude],dims=['time', 'latitude','longitude'])
    dat1=xr.concat([dat, behind], dim='time');dat2=xr.concat([front, dat1], dim='time')
    return dat2
dat=mld*0;dat=xr.DataArray(dat.data, coords=[dat.time,dat.latitude,dat.longitude],dims=['time', 'latitude','longitude'])
MLD=concat_time(mld.data+dat)
MLD,lat,lon,int_U,int_V=MLD,MLD.latitude,MLD.longitude,mlu,mlv

RE=6378000;latarr=lat*np.pi/180.;lonarr=lon*np.pi/180.;

dHtmp=MLD.differentiate('time');
changeH_change_time=0.5*(dHtmp.isel(time=slice(0,-1))+dHtmp.isel(time=slice(1,None)));
#---------------------------------
# 2. Time, horizontal derivatives of MLD
#----------------------------------
dHtmp=(MLD.differentiate('longitude')*np.cos(latarr.isel(latitude=slice(0,None))))/(lonarr.differentiate('longitude'))
dHdx=0.5*(dHtmp.isel(longitude=slice(0,None))+dHtmp.isel(longitude=slice(1,None)))/RE

dHtmp=MLD.differentiate('latitude')/(latarr.differentiate('latitude'))
dHdy=0.5*(dHtmp.isel(latitude=slice(0,None))+dHtmp.isel(latitude=slice(1,None)))/RE;
#---------------------------------
# 3. Entrainment velocity
#----------------------------------
const1=24*3600*30.41
w_entr=changeH_change_time + (int_U*dHdx)*const1+ int_V*dHdy*const1+(He_24.data+dat).differentiate('time')*-1
# do you need to divide changeH_change_time by 86400 like is done in the matlab code?
wsgn=w_entr
wsgn = wsgn.where(wsgn > 0, 0) # wsgn > 0 is true, wsgn < 0 is false, replace false w 0
wsgn = wsgn.where(wsgn == 0, 1) # wsgn=0 is true, not zero is false, replace false with 1
#m/month
dhdt=wsgn*changeH_change_time[:,1:,1:] *(mlt-T_n+mask)/mld*-1
ver_adv=wsgn* ((int_U*dHdx)[:,1:,:]*const1+ int_V*dHdy[:,:,1:]*const1)*(mlt-T_n+mask)/mld*-1
Wh=wsgn*((He_24.data+dat).differentiate('time')*-1)*(mlt-T_n+mask)/mld*-1
VER=(Wh+ver_adv+dhdt)

In [51]:
dat=ild*0;dat=xr.DataArray(dat.data, coords=[dat.time,dat.latitude,dat.longitude],dims=['time', 'latitude','longitude'])
mlt_sum=xr.DataArray(np.swapaxes(np.swapaxes(mlt_sum.data,2,1),2,1),coords=[np.arange(0,13,1),latitude,longitude],dims=['time','latitude','longitude'])
hb_data=xr.Dataset({'ild':ild.data+dat,'mld':mld.data+dat,'mlt':mlt.data+dat,'T_n':T_n.data+dat,'mlu':mlu,'mlv':mlv.data+dat,'Qnet':Qnet.data+dat,
                    'Qpen':Qpen.data+dat,'He_24':He_24.data+dat,'mask':mask.data+dat,'Q':Q.data+dat,'mlt_tend':mlt_sum.diff('time').data+dat,
                    'dhdt':dhdt,'wsgn':wsgn,'ver_adv':ver_adv,'Wh':Wh,'VER':VER})
hb_data.to_netcdf('E:\\data\\oras5\\somali-oras5\\no_cycle_MLD_every_term_climate_1.nc')

### en4

In [ ]:
file="E:\data\en4\EN4_1980-2023_ptemp+salt.nc";print(file)
ds0= xr.open_dataset(file);ds0=ds0.sel(latitude=slice(0,20),longitude=slice(35,65));ds=ds0.salt
en4_salt=ds.groupby('time.month').mean().rename({'month': 'time'});ds=ds0.temp-273.15
en4_ptemp=ds.groupby('time.month').mean().rename({'month': 'time'})
argo_salt=en4_salt;argo_ptemp=en4_ptemp
depth=argo_salt*0+argo_salt.LEV
lat=argo_salt*0+argo_salt.latitude
import seawater as sw
dbar=sw.pres(depth,lat)+argo_salt*0
import gsw
ct=gsw.CT_from_pt(argo_salt, argo_ptemp)
en4_rho,alpha,beta=gsw.rho_alpha_beta(argo_salt, ct, dbar)#rho是密度
rho_mean=en4_rho.mean().data
en4_rho=en4_rho-rho_mean

In [ ]:
ds=en4_rho;en4_mask=function.make_mask3D(en4_rho.sel(LEV=ds.LEV[0]),5/(ds.latitude.data[1]-ds.latitude.data[0]),ds.sel(LEV=ds.LEV[0]))

In [ ]:
ds=en4_ptemp;d_temp=-0.41
sst=ds.sel(LEV=ds.LEV[0])
ds=ds.interp(LEV=np.arange(ds.LEV[0],109.5,0.5))
a=(ds>(sst+d_temp))
ild_idx=ds.where(a,drop=False).squeeze().notnull().argmin(dim='LEV')#
en4_ild=ds.LEV[ild_idx];print('max:',ild.max())
#ds.where(a, drop=True)
i=6;levels=np.arange(0,110,5)
plt.subplot(1,2,1);plt.contourf(en4_ild[i,:,:]+sst[0,:,:]*0,levels=levels);plt.colorbar()
file="E:\data\oras5\somali-oras5\oras5_mld&ild&mlt&T_n&mlu&mlv_climate.nc"
ds0 = xr.open_dataset(file);ild1 = ds0['ild'].sel(latitude=slice(0,20))
plt.subplot(1,2,2);plt.contourf(ild1[i,:,:],levels=levels);plt.colorbar()

In [ ]:
ds=en4_rho;d_rho=0.3
ssrho=ds.sel(LEV=ds.LEV[0])
ds=ds.interp(LEV=np.arange(ds.LEV[0],109.5,0.5))
a=(ds<(ssrho+d_rho))
mld_idx=ds.where(a,drop=False).squeeze().notnull().argmin(dim='LEV')#寻找最大深度，notnull使得缺失值为False，不参与后面的计算
en4_mld=ds.LEV[mld_idx];print('max:',mld.max())
#ds.where(a, drop=True)
i=7;levels=np.arange(0,110,5)
plt.subplot(1,2,1);plt.contourf(en4_mld[i,:,:]+sst[0,:,:]*0,levels=levels);plt.colorbar()
file="E:\data\oras5\somali-oras5\oras5_mld&ild&mlt&T_n&mlu&mlv_climate.nc"
ds0 = xr.open_dataset(file);mld1 = ds0['mld'].sel(latitude=slice(0,20))
plt.subplot(1,2,2);plt.contourf(mld1[i,:,:],levels=levels);plt.colorbar()